In [14]:
import numpy as np, libstempo as LT, json, libstempo.toasim as LTT
import astroabc
import matplotlib
import matplotlib.pyplot as pl
%matplotlib inline

# 1. Define the data here

In [13]:
parpath = "../NANOGrav-DWG/11yr_stochastic_analysis/nano11y_data/partim/B1855+09_NANOGrav_11yv0.gls.par"
timpath = "../NANOGrav-DWG/11yr_stochastic_analysis/nano11y_data/partim/B1855+09_NANOGrav_11yv0.tim"

psr = LT.tempopulsar(parfile = parpath, timfile = timpath)

data = psr.toas()

# 2. Define the simulation here

In [18]:
def simulation(params):
    
    #params should be: psr, white_noise(fixed), red_noise(from noisefile, but will vary)
    psr_copy = psr
    
    LTT.make_ideal(psr_copy)
    
    #add white noise:
    LTT.add_efac(psr_copy, efac = params[1])
    LTT.add_equad(psr_copy, equad = params[2])
    LTT.add_jitter(psr_copy, ecorr = params[3])
    
    #add red noise, to be varied later
    LTT.add_rednoise(psr_copy, params[4], params[5])
    
    return psr_copy.toas()

In [20]:
simulation([psr, 1., 1., 1., 1., 1.])

array([53398.30332695, 53398.30331807, 53398.30331855, ...,
       57415.30852476, 57415.30854268, 57415.30852736], dtype=float128)

In [21]:
data

array([53358.72366849, 53358.72366849, 53358.72366849, ...,
       57375.72889084, 57375.72889066, 57375.72889078], dtype=float128)